In [ ]:
# Lung Cancer Detection using CNNs
This project uses CNN models for detecting lung cancer from CT scan images. It includes data preprocessing, augmentation, model architecture, training, and evaluation.


In [ ]:
#2. Imports and Setup
python
Copy code
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import Activation, AveragePooling2D, Concatenate, Add, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import os
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#3. Define Hyperparameters
python
Copy code
image_height, image_width = 224, 224
batch_size = 32
epochs = 20
train_dir = "/path/to/train"
val_dir = "/path/to/validation"
test_dir = "/path/to/test"


In [ ]:
#4. Data Augmentation and Generators
python
Copy code
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical"
)

validation_data = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
#5. Define Identity and Convolutional Blocks
python
Copy code
def identity_block(input_tensor, kernel_size, filters):
    filters1, filters2, filters3 = filters
    x = Conv2D(filters1, (1, 1), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters3, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    x = Conv2D(filters1, (1, 1), strides=strides)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters3, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    shortcut = Conv2D(filters3, (1, 1), strides=strides)(input_tensor)
    shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x
#6. Define ResNet-like Model and Custom Model
python
Copy code
def reuse_learning(input_shape=(224, 224, 3), include_top=False):
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    # Residual blocks
    x = conv_block(x, 3, [64, 64, 256], strides=(1, 1))
    for _ in range(2):
        x = identity_block(x, 3, [64, 64, 256])
    
    x = conv_block(x, 3, [128, 128, 512])
    for _ in range(3):
        x = identity_block(x, 3, [128, 128, 512])
    
    x = conv_block(x, 3, [256, 256, 1024])
    for _ in range(5):
        x = identity_block(x, 3, [256, 256, 1024])
    
    x = conv_block(x, 3, [512, 512, 2048])
    for _ in range(2):
        x = identity_block(x, 3, [512, 512, 2048])
    
    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(1000, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(4, activation='softmax')(x)

    model = Model(inputs=img_input, outputs=x)
    return model

def chest_custom_model1(input_shape=(224, 224, 3)):
    base_resnet = reuse_learning(input_shape, include_top=False)
    
    for layer in base_resnet.layers[:150]:
        layer.trainable = False
    
    inputs = Input(shape=input_shape)
    x = base_resnet(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax', kernel_regularizer=l2(0.01))(x)
    model = Model(inputs=inputs, outputs=output)
    return model
#7. Model Fusion, Compilation, and Callbacks
python
Copy code
def fuse_models(resnet_model, chest_custom_model1, input_shape=(224, 224, 3)):
    input_layer = Input(shape=input_shape)
    features1 = resnet_model(input_layer)
    features2 = chest_custom_model1(input_layer)
    flattened_features1 = Flatten()(features1)
    flattened_features2 = Flatten()(features2)
    combined_features = Concatenate()([flattened_features1, flattened_features2])
    x = Dense(1024, activation='relu')(combined_features)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

model1 = reuse_learning(input_shape=(224, 224, 3), include_top=False)
model2 = chest_custom_model1(input_shape=(224, 224, 3))
model = fuse_models(model1, model2)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-4)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
#8 Training and Evaluation
python
Copy code
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=validation_data,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")